In [1]:
import os
import pandas as pd

In [2]:
delta_table_path = "./../data/delta"
entities = ([file.split("=")[-1] for file in os.listdir(delta_table_path)])
df = pd.read_parquet(delta_table_path, filters=[("entity", "=", entities[0])])
target = df.target
df["entity"] = df["entity"].astype(object)
df = df.drop(columns=["target", "entity"])
print(df.shape)
df.head()

(980, 20)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-1.079897,-0.521745,-1.323366,-2.598829,-0.322770,-0.098911,0.896676,-1.218417,-1.898309,-1.756554,1.545330,-0.880667,-0.687532,0.036406,-0.960325,1.722097,-0.318500,0.286957,-2.099939,0.143611
1,-1.096871,0.580129,0.921091,0.697789,0.978352,-0.814747,0.739980,-0.149653,-0.088223,-0.681317,1.212889,1.911099,-0.780296,-0.298103,0.218609,-0.619019,2.251253,-1.023148,1.010799,1.011284
2,0.463016,0.626208,-0.246439,0.035407,-0.472789,-0.771598,-0.012589,0.988110,2.118438,-1.327387,-1.280916,0.518309,0.473234,-1.689038,-2.188064,0.592483,0.063437,1.259331,0.660135,-2.306905
3,-0.596635,-1.618210,-1.667084,1.090140,-0.018677,-0.806299,0.264438,-0.648286,-1.072946,0.107288,-1.069770,-1.396121,-0.490691,1.500925,1.084174,0.794798,-0.945093,-0.054545,-1.181674,-0.291081
4,-1.562458,0.740043,0.288696,-2.440389,1.064151,-0.830998,0.844438,-0.567137,-1.790544,-0.597115,1.761643,-0.672102,-1.212106,-0.021069,1.777037,0.395582,0.454735,0.356403,-2.036692,-0.049178


In [3]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, target,
                                                    train_size=0.75, test_size=0.25, random_state=42)

tpot = TPOTClassifier(generations=2, population_size=3, verbosity=2, random_state=42)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
# tpot.export('tpot_mlscale_model.py')

Optimization Progress:   0%|          | 0/9 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9102040816326531

Generation 2 - Current best internal CV score: 0.9102040816326531

Best pipeline: GradientBoostingClassifier(MinMaxScaler(input_matrix), learning_rate=0.01, max_depth=5, max_features=0.45, min_samples_leaf=5, min_samples_split=19, n_estimators=100, subsample=1.0)
0.9469387755102041


In [4]:
# import ray
# import ray.data
# # delta_table_path = "./../data/delta/entity=entity_0/part-00001-0c46d1c5-5bab-42e2-aa2e-5ef32f634cc5-c000.snappy.parquet"
# ds = ray.data.read_parquet(delta_table_path, partition_filter=lambda d: d["entity"] == entities[0]) # 
# # ds